In [1]:
!pip install kaggle
!kaggle datasets download -d hasyimabdillah/workoutfitness-video


Dataset URL: https://www.kaggle.com/datasets/hasyimabdillah/workoutfitness-video
License(s): CC-BY-NC-SA-4.0
  0%|                                      | 11.0M/4.32G [00:08<53:52, 1.43MB/s]^C
  0%|                                      | 11.0M/4.32G [00:08<58:31, 1.32MB/s]
User cancelled operation


In [2]:
import zipfile
import os

# İndirilen zip dosyasını açma
with zipfile.ZipFile('workoutfitness-video.zip', 'r') as zip_ref:
    zip_ref.extractall('workout_videos')

# Dosya yapısını inceleme
for root, dirs, files in os.walk('workout_videos'):
    for file in files:
        print(os.path.join(root, file))


workout_videos/deadlift/deadlift_17.mp4
workout_videos/deadlift/deadlift_16.mp4
workout_videos/deadlift/deadlift_14.mp4
workout_videos/deadlift/deadlift_28.mp4
workout_videos/deadlift/deadlift_29.mp4
workout_videos/deadlift/deadlift_15.mp4
workout_videos/deadlift/deadlift_11.mp4
workout_videos/deadlift/deadlift_10.mp4
workout_videos/deadlift/deadlift_12.mp4
workout_videos/deadlift/deadlift_13.mp4
workout_videos/deadlift/deadlift_9.mp4
workout_videos/deadlift/deadlift_8.mp4
workout_videos/deadlift/deadlift_1.mp4
workout_videos/deadlift/deadlift_3.mp4
workout_videos/deadlift/deadlift_2.mp4
workout_videos/deadlift/deadlift_6.mp4
workout_videos/deadlift/deadlift_7.mp4
workout_videos/deadlift/deadlift_5.mp4
workout_videos/deadlift/deadlift_4.mp4
workout_videos/deadlift/deadlift_22.mp4
workout_videos/deadlift/deadlift_23.mp4
workout_videos/deadlift/deadlift_21.mp4
workout_videos/deadlift/deadlift_20.mp4
workout_videos/deadlift/deadlift_18.mp4
workout_videos/deadlift/deadlift_30.mp4
workout_v

In [3]:
import os
import cv2

# Video klasörünü belirleme
video_dir = 'workout_videos'
frame_dir = 'frames'

# Hareket sınıfları
classes = ['deadlift', 'hammer curl', 'tricep Pushdown', 'squat', 'push-up',
           'tricep dips', 'lat pulldown', 'barbell biceps curl', 'chest fly machine',
           'incline bench press', 'leg extension', 'shoulder press', 't bar row',
           'decline bench press', 'bench press', 'lateral raise', 'pull Up', 'plank',
           'leg raises', 'hip thrust', 'romanian deadlift', 'russian twist']

# Sınıf klasörlerini oluşturma
for class_name in classes:
    os.makedirs(os.path.join(frame_dir, class_name), exist_ok=True)

# Her video dosyasını kontrol etme ve frame'leri uygun sınıfa kaydetme
for class_name in classes:
    class_path = os.path.join(video_dir, class_name)
    for video_file in os.listdir(class_path):
        if video_file.endswith(('.mp4', '.MOV')):
            video_path = os.path.join(class_path, video_file)
            cap = cv2.VideoCapture(video_path)
            frame_count = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frame_filename = f"{class_name}_{video_file.split('.')[0]}_{frame_count}.jpg"
                frame_path = os.path.join(frame_dir, class_name, frame_filename)
                cv2.imwrite(frame_path, frame)
                frame_count += 1
            cap.release()


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Veri üreticisi ile veri ön işleme
datagen = ImageDataGenerator(validation_split=0.2)

train_generator = datagen.flow_from_directory(
    'frames',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    'frames',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Eğitim veri setinizdeki sınıf sayısını kontrol etme
num_classes = len(train_generator.class_indices)
print(f"Number of classes in the training set: {num_classes}")
print(f"Class indices: {train_generator.class_indices}")


Found 116036 images belonging to 22 classes.
Found 29000 images belonging to 22 classes.
Number of classes in the training set: 22
Class indices: {'barbell biceps curl': 0, 'bench press': 1, 'chest fly machine': 2, 'deadlift': 3, 'decline bench press': 4, 'hammer curl': 5, 'hip thrust': 6, 'incline bench press': 7, 'lat pulldown': 8, 'lateral raise': 9, 'leg extension': 10, 'leg raises': 11, 'plank': 12, 'pull Up': 13, 'push-up': 14, 'romanian deadlift': 15, 'russian twist': 16, 'shoulder press': 17, 'squat': 18, 't bar row': 19, 'tricep Pushdown': 20, 'tricep dips': 21}


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

model = Sequential([
    Input(shape=(64, 64, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')  # Sınıf sayısını doğru ayarlayın
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs=10, validation_data=validation_generator)


Epoch 1/10
3627/3627 ━━━━━━━━━━━━━━━━━━━━ 1495s 411ms/step - accuracy: 0.8980 - loss: 2.1267 - val_accuracy: 0.3993 - val_loss: 10.1300
Epoch 2/10
3627/3627 ━━━━━━━━━━━━━━━━━━━━ 1508s 415ms/step - accuracy: 0.9911 - loss: 0.0474 - val_accuracy: 0.3736 - val_loss: 18.1065
Epoch 3/10
3627/3627 ━━━━━━━━━━━━━━━━━━━━ 1498s 412ms/step - accuracy: 0.9955 - loss: 0.0335 - val_accuracy: 0.3763 - val_loss: 30.6542
Epoch 4/10
3627/3627 ━━━━━━━━━━━━━━━━━━━━ 1513s 417ms/step - accuracy: 0.9958 - loss: 0.0475 - val_accuracy: 0.3868 - val_loss: 41.5365
Epoch 5/10
3627/3627 ━━━━━━━━━━━━━━━━━━━━ 1714s 472ms/step - accuracy: 0.9972 - loss: 0.0460 - val_accuracy: 0.3936 - val_loss: 58.6276
Epoch 6/10
3627/3627 ━━━━━━━━━━━━━━━━━━━━ 1547s 426ms/step - accuracy: 0.9975 - loss: 0.0485 - val_accuracy: 0.4269 - val_loss: 87.8194
Epoch 7/10
3627/3627 ━━━━━━━━━━━━━━━━━━━━ 1546s 426ms/step - accuracy: 0.9980 - loss: 0.0468 - val_accuracy: 0.3831 - val_loss: 103.7243
Epoch 8/10
3627/3627 ━━━━━━━━━━━━━━━━━━━━ 1670s

In [11]:
import json

# Modeli kaydetme
model.save('exercise_detection_model_corrected.h5')

# Sınıf isimlerini kaydetme
class_indices = train_generator.class_indices
class_names = {v: k for k, v in class_indices.items()}

with open('class_names.json', 'w') as f:
    json.dump(class_names, f)


In [9]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Modeli yükleme
model = load_model('exercise_detection_model_corrected.h5')

# Sınıf isimlerini manuel olarak tanımlama (num_classes'a göre güncelleyin)
class_names = {v: k for k, v in train_generator.class_indices.items()}

# Yeni video dosya yolu
new_video_path = 'workout_videos/bench press/bench press_10.mp4'

# Video okuma
cap = cv2.VideoCapture(new_video_path)
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Çerçeveyi ön işleme tabi tutarak model için uygun hale getirme
    processed_frame = cv2.resize(frame, (64, 64))
    processed_frame = processed_frame.astype('float32') / 255.0
    processed_frame = np.expand_dims(processed_frame, axis=0)

    # Modelin tahmini
    predictions = model.predict(processed_frame)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    confidence = np.max(predictions)
    print(f"Predicted Class Index: {predicted_class_index}, Confidence: {confidence}")

    # Tahmin edilen hareketin adını ve doğruluk oranını yazdırma
    if predicted_class_index in class_names:
        print(f"Frame {frame_count}:")
        print(f"Predicted Exercise: {class_names[predicted_class_index]}, Confidence: {confidence:.2f}")
    else:
        print(f"Frame {frame_count}: Predicted class {predicted_class_index} not found in class names.")

    frame_count += 1

cap.release()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Class Index: 1, Confidence: 0.7117964029312134
Frame 0:
Predicted Exercise: bench press, Confidence: 0.71
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Predicted Class Index: 1, Confidence: 0.7150495052337646
Frame 1:
Predicted Exercise: bench press, Confidence: 0.72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Predicted Class Index: 1, Confidence: 0.7062633633613586
Frame 2:
Predicted Exercise: bench press, Confidence: 0.71
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Predicted Class Index: 1, Confidence: 0.6980128884315491
Frame 3:
Predicted Exercise: bench press, Confidence: 0.70
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Predicted Class Index: 1, Confidence: 0.7040899395942688
Frame 4:
Predicted Exercise: bench press, Confidence: 0.70
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Predicted Class Index: 1, Confidence: 0.7095310688018799
Frame 5:
Predicted Exercise: bench press, Confidence: 0.71
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Predicted Class Index: 1, Confidence: 0.7089

In [10]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from collections import Counter

# Modeli yükleme
model = load_model('exercise_detection_model_corrected.h5')

# Sınıf isimlerini manuel olarak tanımlama
class_names = {v: k for k, v in train_generator.class_indices.items()}

# Yeni video dosya yolu
new_video_path = 'workout_videos/bench press/bench press_10.mp4'

# Video okuma
cap = cv2.VideoCapture(new_video_path)
frame_count = 0

# Tahmin sonuçlarını saklamak için liste
predictions_list = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Çerçeveyi ön işleme tabi tutarak model için uygun hale getirme
    processed_frame = cv2.resize(frame, (64, 64))
    processed_frame = processed_frame.astype('float32') / 255.0
    processed_frame = np.expand_dims(processed_frame, axis=0)

    # Modelin tahmini
    predictions = model.predict(processed_frame)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    confidence = np.max(predictions)

    # Tahmin sonuçlarını listeye ekleme
    predictions_list.append(predicted_class_index)

    frame_count += 1

cap.release()

# En sık görülen sınıfı bulma
most_common_prediction = Counter(predictions_list).most_common(1)[0][0]
most_common_confidence = np.mean([predictions.max() for predictions in predictions_list])

print(f"Predicted Exercise: {class_names[most_common_prediction]}, Confidence: {most_common_confidence:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/ste